In [20]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('text.csv')

In [5]:
df.head(10)

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
5,5,i was beginning to feel quite disheartened,0
6,6,i would think that whomever would be lucky eno...,2
7,7,i fear that they won t ever feel that deliciou...,1
8,8,im forever taking some time out to have a lie ...,5
9,9,i can still lose the weight without feeling de...,0


In [6]:
df.columns

Index(['Unnamed: 0', 'text', 'label'], dtype='object')

In [7]:
df.drop(columns='Unnamed: 0', inplace=True)

In [8]:
print(f'The Shape Of Data Is : {df.shape}')

The Shape Of Data Is : (416809, 2)


In [9]:
# checking the duplicates we have found some so we are going to remove them 
df.duplicated().sum()

686

In [10]:
# removing the duplicates
df = df.drop_duplicates()

In [11]:
# rechecking the duplicates
df.duplicated().sum()

0

In [14]:
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

df.head()

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [21]:
stop = stopwords.words('english')
df["text"] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df.head()

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\hassa/nltk_data'
    - 'C:\\Users\\hassa\\anaconda3\\nltk_data'
    - 'C:\\Users\\hassa\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\hassa\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\hassa\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
